## 0. Import and set the path

In [80]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, f1_score
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

#--------------------------------------------------

import kagglehub

# Download latest version
path = kagglehub.dataset_download("asjad99/mimiciii")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/mimiciii


## 1. Import data

In [81]:
df_PATIENTS = pd.read_csv('/kaggle/input/mimiciii/mimic-iii-clinical-database-demo-1.4/PATIENTS.csv')
df_CHARTEVENTS = pd.read_csv('/kaggle/input/mimiciii/mimic-iii-clinical-database-demo-1.4/CHARTEVENTS.csv')
df_ADMISSIONS = pd.read_csv('/kaggle/input/mimiciii/mimic-iii-clinical-database-demo-1.4/ADMISSIONS.csv')
df_D_ITEMS = pd.read_csv('/kaggle/input/mimiciii/mimic-iii-clinical-database-demo-1.4/D_ITEMS.csv')

df_ADMISSIONS = df_ADMISSIONS[df_ADMISSIONS['subject_id'] != 10120]
df_PATIENTS = df_PATIENTS[df_PATIENTS['subject_id'] != 10120]

df_PATIENTS = df_PATIENTS.loc[:,['subject_id','gender']]

## 2. Eliminate some columns

In [82]:
df_ADMISSIONS.drop(columns=['admittime', 'deathtime', 'dob'], inplace=True, errors='ignore')

In [83]:
print(df_PATIENTS.columns)
print(df_CHARTEVENTS.columns)
print(df_ADMISSIONS.columns)

Index(['subject_id', 'gender'], dtype='object')
Index(['row_id', 'subject_id', 'hadm_id', 'icustay_id', 'itemid', 'charttime',
       'storetime', 'cgid', 'value', 'valuenum', 'valueuom', 'warning',
       'error', 'resultstatus', 'stopped'],
      dtype='object')
Index(['row_id', 'subject_id', 'hadm_id', 'dischtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'religion', 'marital_status', 'ethnicity', 'edregtime', 'edouttime',
       'diagnosis', 'hospital_expire_flag', 'has_chartevents_data'],
      dtype='object')


## 3. Missing Value Removal

In [67]:
# 'value' 컬럼 숫자형으로 변환
df_CHARTEVENTS['value'] = pd.to_numeric(df_CHARTEVENTS['value'], errors='coerce')

# 같은 subject_id의 평균값으로 NaN 채우기
df_CHARTEVENTS['value'] = df_CHARTEVENTS.groupby('subject_id')['value'].transform(
    lambda x: x.fillna(x.mean())
)

# 만약 해당 subject_id의 모든 값이 NaN이면 전체 평균으로 대체
df_CHARTEVENTS['value'].fillna(df_CHARTEVENTS['value'].mean(), inplace=True)

# 결과 확인
print(df_CHARTEVENTS.isna().sum())

row_id               0
subject_id           0
hadm_id              0
icustay_id          81
itemid               0
charttime            0
storetime            0
cgid                 0
value                0
valuenum        434471
valueuom        518500
warning         376076
error           376076
resultstatus    736681
stopped         383706
dtype: int64


In [68]:
'''
# subject_id별 평균으로 NaN 채우기
df_CHARTEVENTS['value'] = df_CHARTEVENTS.groupby('subject_id')['value'].transform(
    lambda x: x.fillna(x.mean())
)

# 그래도 NaN이면 전체 평균으로 대체
df_CHARTEVENTS['value'].fillna(df_CHARTEVENTS['value'].mean(), inplace=True)
'''

"\n# subject_id별 평균으로 NaN 채우기\ndf_CHARTEVENTS['value'] = df_CHARTEVENTS.groupby('subject_id')['value'].transform(\n    lambda x: x.fillna(x.mean())\n)\n\n# 그래도 NaN이면 전체 평균으로 대체\ndf_CHARTEVENTS['value'].fillna(df_CHARTEVENTS['value'].mean(), inplace=True)\n"

In [86]:
# 1) (선택) 분석할 itemid만 필터 – 팀원 예시: 211, 618, 646
vital_ids = {211: 'Heart Rate', 618: 'Resp Rate', 646: 'SpO2'}
df_vitals = df_CHARTEVENTS[df_CHARTEVENTS['itemid'].isin(vital_ids.keys())].copy()

# 2) value 숫자형 & subject_id별 평균으로 1차 NaN 채우기
df_vitals['value'] = pd.to_numeric(df_vitals['value'], errors='coerce')
df_vitals['value'] = (
    df_vitals
    .groupby(['subject_id','itemid'])['value']
    .transform(lambda s: s.fillna(s.mean()))
)
# 3) 그래도 NaN이면 전체 평균으로 2차 채우기
df_vitals['value'].fillna(df_vitals['value'].mean(), inplace=True)

# 4) pivot (subject_id × itemid 평균)
df_pivot = (
    df_vitals
    .pivot_table(index='subject_id',
                 columns='itemid',
                 values='value',
                 aggfunc='mean')
    .reset_index()
    .rename(columns=vital_ids)               # 숫자 대신 라벨 붙이기
)

# 5) pivot 뒤 남은 NaN을 컬럼별 평균으로 채우기
for col in df_pivot.columns[1:]:             # subject_id 제외
    df_pivot[col].fillna(df_pivot[col].mean(), inplace=True)

# 6) Admissions + Patients 병합
df_adm_pat = (
    df_ADMISSIONS.drop(columns=['admittime','deathtime','dob'], errors='ignore')
    .merge(df_PATIENTS[['subject_id','gender']], on='subject_id', how='left')
)

# 7) 최종 병합
df_final = df_adm_pat.merge(df_pivot, on='subject_id', how='left')

# 8) 숫자형 컬럼별 평균으로 한 번 더 NaN 정리
for col in df_final.select_dtypes(include='number').columns:
    df_final[col].fillna(df_final[col].mean(), inplace=True)

In [89]:
# 숫자형 컬럼만 선택
numeric_cols = df_final.select_dtypes(include='number').columns

# 각 컬럼별 평균값으로 NaN 채우기
for col in numeric_cols:
    mean_val = df_final[col].mean()
    df_final[col].fillna(mean_val, inplace=True)

In [91]:
pd.set_option('display.max_rows', None)  # 모든 행 출력
df_final

,row_id,subject_id,hadm_id,dischtime,admission_type,admission_location,discharge_location,insurance,language,religion,...,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,gender,Heart Rate,Resp Rate,SpO2
0,12258,10006,142345,2164-11-01 17:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,...,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1,F,82.000000,25.375000,98.368421
1,12263,10011,105331,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,CATHOLIC,...,UNKNOWN/NOT SPECIFIED,NaN,NaN,HEPATITIS B,1,1,F,82.398649,15.447458,98.989691
2,12265,10013,165520,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,...,UNKNOWN/NOT SPECIFIED,NaN,NaN,SEPSIS,1,1,F,92.146341,21.487805,91.000000
3,12269,10017,199207,2149-06-03 18:42:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,...,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1,F,91.836735,26.479167,93.625000
4,12270,10019,177759,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,...,WHITE,NaN,NaN,ALCOHOLIC HEPATITIS,1,1,M,83.333333,30.441860,90.166667
5,12277,10026,103770,2195-05-24 11:45:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,OTHER,...,WHITE,2195-05-17 01:49:00,2195-05-17 08:29:00,STROKE/TIA,0,1,F,70.273973,13.931507,98.071429
6,12278,10027,199395,2190-07-25 14:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Medicare,NaN,CATHOLIC,...,WHITE,NaN,NaN,MITRAL REGURGITATION;CORONARY ARTERY DISEASE\...,0,1,F,88.164251,23.728155,96.685990
7,12280,10029,132349,2139-10-02 14:29:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,PROTESTANT QUAKER,...,WHITE,2139-09-22 06:03:00,2139-09-22 11:50:00,SYNCOPE;TELEMETRY,0,1,M,97.864865,24.418919,97.254545
8,12282,10032,140372,2138-04-15 14:35:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,CATHOLIC,...,WHITE,2138-04-02 14:56:00,2138-04-02 20:40:00,RIGHT HUMEROUS FRACTURE,0,1,M,92.770833,20.851064,96.354167
9,12283,10033,157235,2132-12-08 15:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,POLI,CATHOLIC,...,WHITE,2132-12-04 20:11:00,2132-12-05 04:05:00,RENAL FAILIURE-SYNCOPE-HYPERKALEMIA,0,1,F,88.375000,17.416667,94.958333
